# 🧪 PIP INSTALL
Instalar pacotes: Autodistill, GroundedSAM, YOLOv8, Supervision e Roboflow


In [ ]:
!pip install -q \
autodistill \
autodistill-grounded-sam \
autodistill-yolov8 \
supervision==0.9.0 \
roboflow

# 🗂️ IMPORT
**NOTE**: Bibliotecas principais como autodistill (CaptionOntology, GroundedSAM e YOLOv8) para lidar com rotulagem automatica e detecção de objetos


In [ ]:
import os
import cv2
import supervision as sv
from autodistill.detection import CaptionOntology
from autodistill_grounded_sam import GroundedSAM
from autodistill_yolov8 import YOLOv8
from IPython.display import Image

# 🖼️ PREPARANDO DADOS

### Carregando imagens

Definindo o diretorio atual e vizualizando

In [ ]:
HOME = os.getcwd()
print(HOME)

Cria um diretório chamado `images` no dict `HOME`, para carregar as imagens atraves do GitHub


In [ ]:
!mkdir {HOME}/images
!git clone https://github.com/vitorAugusto2/tcc-a2d2.git
!mv {HOME}/tcc-a2d2/images/* {HOME}/images/
!rm -rf {HOME}/tcc-a2d2

### Exibir amostra de imagem nao rotulada

In [ ]:
IMAGE_DIR_PATH = f"{HOME}/images"

image_paths = sv.list_files_with_extensions(
    directory=IMAGE_DIR_PATH,
    extensions=["png"])

print(f"Quantidades de imagens: {len(image_paths)}")

In [ ]:
SAMPLE_SIZE = 16
SAMPLE_GRID_SIZE = (4, 4)
SAMPLE_PLOT_SIZE = (16, 16)

titles = [
    image_path.stem
    for image_path
    in image_paths[:SAMPLE_SIZE]]
images = [
    cv2.imread(str(image_path))
    for image_path
    in image_paths[:SAMPLE_SIZE]]

sv.plot_images_grid(images=images, titles=titles, grid_size=SAMPLE_GRID_SIZE, size=SAMPLE_PLOT_SIZE)

# 🔪 AUTODISTILL: ROTULO AUTOMATICO

### Definir ontologia

**Ontologia** - uma Ontologia define como seu Modelo Base é solicitado, o que seu Conjunto de Dados descreverá e o que seu Modelo Alvo irá prever. Uma ontologia simples é a CaptionOntology que solicita um modelo base com legendas de texto e as mapeia para nomes de classes.

In [ ]:
"""
  Ontology: como o modelo base é solicitado
  Descreve o que voce realmente ve no conjunto de dados e no que deseja treinar
  seu modelo destino

  Lancar os prompts e classes para gerar as mascaras de segmentacao
    -> (prompt : class)
"""
"""
  Classificar as classes de interesse, exemplo:
  ```python
    ontology=CaptionOntology({
      "car": "car",
      "pedestrian": "pedestrian",
      "tree": "tree"
    })
  ```
"""

ontology=CaptionOntology({
  # digite (prompt : class)
})

### Iniciar modelo base e rótulo automático

**Ontologia** - define como seu Modelo Base é solicitado, o que seu Conjunto de Dados descreverá e o que seu Modelo Alvo irá prever. Uma ontologia simples é aquela CaptionOntologyque solicita um modelo base com legendas de texto e os mapeia para nomes de classes.
- Uma ontologia em Computação é um modelo de dados que representa um conjunto de conceitos dentro de um domínio e os relacionamentos entre estes.

In [ ]:
DATASET_DIR_PATH = f"{HOME}/dataset" # diretorio de treinamento do dataset

In [ ]:
# Modelo base é iniciado por ontology
base_model = GroundedSAM(ontology=ontology) # modelo base: GroundedSAM
dataset = base_model.label(
    input_folder=IMAGE_DIR_PATH,      # entrada de dados
    extension=".png",                 # extensao da imagem .png
    output_folder=DATASET_DIR_PATH)   # armazena a saida dos dados rotulados

### Exibir amostra do conjunto de dados

**Conjunto de dados/dataset** - dados rotulados automaticamente que pode ser usado para treinar um modelo de destino, como por exemplo YOLOv8. É a saída gerada por um Modelo Base.

In [ ]:
ANNOTATIONS_DIRECTORY_PATH = f"{HOME}/dataset/train/labels" # dados de rotulagem das imagens treinadas
IMAGES_DIRECTORY_PATH = f"{HOME}/dataset/train/images"      # imagens de treino rotuladas
DATA_YAML_PATH = f"{HOME}/dataset/data.yaml"                # arquivo formato YOLO (.yaml -> id, x, y, largura, altura)

**NOTA**: a quantidade de imagens total é dividida em `train` e `valid`

In [ ]:
dataset = sv.DetectionDataset.from_yolo(
    images_directory_path=IMAGES_DIRECTORY_PATH,
    annotations_directory_path=ANNOTATIONS_DIRECTORY_PATH,
    data_yaml_path=DATA_YAML_PATH)

len(dataset) # quantidade de imagens de treinamento

In [ ]:
"""
  Exemplos de imagens; Loop for em execucao com todos os parametros que criam
  essa imagens de anotacao com mascara de segmentacao e anotador de caixa de
  supervisao; e tambem dados demograficos para tracar nossa grade de imagens
"""

SAMPLE_SIZE = 16
SAMPLE_GRID_SIZE = (4, 4)
SAMPLE_PLOT_SIZE = (16, 16)

image_names = list(dataset.images.keys())[:SAMPLE_SIZE]

mask_annotator = sv.MaskAnnotator()
box_annotator = sv.BoxAnnotator()

images = []
for image_name in image_names:
    image = dataset.images[image_name]
    annotations = dataset.annotations[image_name]
    labels = [
        dataset.classes[class_id]
        for class_id
        in annotations.class_id]
    annotates_image = mask_annotator.annotate(
        scene=image.copy(),
        detections=annotations)
    annotates_image = box_annotator.annotate(
        scene=annotates_image,
        detections=annotations,
        labels=labels)
    images.append(annotates_image)

sv.plot_images_grid(
    images=images,
    titles=image_names,
    grid_size=SAMPLE_GRID_SIZE,
    size=SAMPLE_PLOT_SIZE)

# 🚘YOLOV8


### Treinamento do modelo
**Modelo de destino** – é um modelo supervisionado que consome um conjunto de dados e gera um modelo destilado que está pronto para implantação. Os modelos de destino geralmente são pequenos, rápidos e ajustados para executar muito bem uma tarefa específica (mas não generalizam muito além das informações descritas em seu conjunto de dados).

**NOTA**: A interface de linha de comando YOLO (CLI) permite comandos simples de linha única sem a necessidade de um ambiente Python. CLI não requer personalização ou código Python . Podes simplesmente executar todas as tarefas a partir do terminal com o comando yolo comanda.

Ultralytics yolo utiliza a seguinte sintaxe:
```python
  yolo TASK MODE ARGS

  Where   TASK (optional) is one of [detect, segment, classify]
          MODE (required) is one of [train, val, predict, export, track]
          ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
```

### Validando o modelo de treinamento
Fornece um conjunto robusto de ferramentas e métricas para avaliar o desempenho dos teus modelos de deteção de objectos
- Precisão e Ajuste de hiperparâmetros

**NOTA**: Os arquivos de treinamento foram gerados e armazenados no diretorio `runs`.

In [ ]:
%cd {HOME}

target_model = YOLOv8("yolov8n.pt")
target_model.train(DATA_YAML_PATH, epochs=5) # define quantas epochs

### Exibir resultados de treinamento e validacao
**NOTA**: Os arquivos dos resultados de treinamento e validacao foram gerados e armazenados no diretorio `runs\detect\trainx`, onde x é numero de 1 a n.

**Matriz de confusao**

In [ ]:
Image(filename=f'{HOME}/runs/detect/trainx/confusion_matrix.png', width=600)

**Metricas de treinamento**

In [ ]:
Image(filename=f'{HOME}/runs/detect/trainx/results.png', width=600)

**Imagens validadas**

In [ ]:
Image(filename=f'{HOME}/runs/detect/trainx/val_batch0_labels.jpg', width=600)